In [38]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.reset_default_graph()


In [39]:
input_data

<module 'tensorflow.examples.tutorials.mnist.input_data' from '/opt/conda/lib/python3.6/site-packages/tensorflow/examples/tutorials/mnist/input_data.py'>

In [40]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [41]:
lr = 0.001
training_iters = 100000
batch_size = 128

In [42]:
n_inputs = 28 # input 28 samples/sentences each time
n_steps = 28 # each input has 28 time steps/ words
n_hidden_units = 128
n_classes = 10

In [43]:
x = tf.placeholder(tf.float32, [None, n_inputs, n_steps])
y = tf.placeholder(tf.float32, {None, n_classes})

In [44]:
weights = {
    'in': tf.Variable(tf.random_normal([n_steps, n_hidden_units])),
    'out': tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=(n_hidden_units,))),
    'out': tf.Variable(tf.constant(0.1, shape=(n_classes,)))
}

In [45]:
def RNN(x, weights, biases):
    # (128batch, 28input, 28time step) -> (128*28, 28time_steps)
    x = tf.reshape(x, [-1,n_steps])
    x_in = tf.matmul(x, weights['in']) + biases['in']
    x_in = tf.reshape(x_in, [-1, n_inputs, n_hidden_units])
    
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units, forget_bias=1.0, state_is_tuple=True)
    # state_is_tuple (last memory, last output), c[t-1], h[t-1]
    _init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)
    
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, x_in, initial_state=_init_state, time_major=False, )
    
    results = tf.matmul(states[1], weights['out']) + biases['out']

    
    return results
    
    
    

In [46]:
pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.initialize_all_variables()
with tf.Session() as sess:
        
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_inputs, n_steps])
        sess.run(train_op, feed_dict={x:batch_xs, y:batch_ys})
        if step % 20 ==0:
            print(sess.run(accuracy, feed_dict={x:batch_xs, y:batch_ys}))
        step += 1

0.1015625
0.7734375
0.8125
0.828125
0.8359375
0.859375
0.8046875
0.859375
0.8671875
0.890625
0.8984375
0.9140625
0.9296875
0.8984375
0.90625
0.9453125
0.9296875
0.953125
0.9453125
0.921875
0.9375
0.9296875
0.953125
0.953125
0.96875
0.9765625
0.9765625
0.9609375
0.96875
0.96875
0.9609375
0.9921875
0.9609375
0.984375
0.9765625
0.96875
0.9765625
0.9609375
0.9921875
0.9765625
